# Traffic example with time-dependent external velocity

Import the necessary packages.

In [ ]:
using ConservationLawsParticles
using RecursiveArrayTools, DifferentialEquations, Plots

## Define the model

- The external velocity is constant and directed to the right.
- The interaction is repulsive at small scales and attractive at long range.
- The mobility models total congestion when the density reaches $1$.

$$
\begin{aligned}
V(x) &= [1 + 0.2\sin(t)] \cdot [1 + 0.2\cos(3x+t)] \\
W(x) &= \frac1{(|x|+1)^2} - \frac1{|x|+1} + 0.02x^2 \\
\mathop{\mathrm{mob}}(\rho) &= (1-\rho)_+^2
\end{aligned}
$$

In [ ]:
V(t, x) = (1 + 0.2sin(t)) * (1 + 0.2cos(3x + t))
@time_independent W(x) = 1/(abs(x)+1)^2 - 1/(abs(x)+1) + .05x^2
mob(ρ) = max(1 - ρ, 0)^2

model = IntegratedModel((V,), ((W,),), (mob,))

plot(W, -4, 4, title="Interaction", label="W")

## Define the ODE problem and the initial conditions

The initial condition approximates $\rho_0 = 1_{[-1,-1/2]} + 1_{[1/2,1]}$.

In [ ]:
n = 25
x0 = ArrayPartition(vcat(range(-1, -.5, length=n), range(.5, 1, length=n)))

tspan = (0., 20.)

prob = ODEProblem(velocities_gen!, x0, tspan, model)

## Solve the ODE

In [ ]:
abstol, reltol = 1e-7, 1e-7

@time sol = solve(prob, BS5(); abstol=abstol, reltol=reltol)

length(sol)

In [ ]:
plot(title="Traffic trajectories", legend=false)
plot!(sol; color=:blue)
plot!(xlabel="time", ylabel="space")
savefig("plots/trajectories2.png")
plot!()

## Plot the density

Let us first compute a more refined solution.

In [ ]:
n = 201
x0 = ArrayPartition(vcat(range(-1, -.5, length=n), range(.5, 1, length=n)))
tspan = (0, 20.)
prob = ODEProblem(velocities_gen!, x0, tspan, model)

abstol, reltol = .5e-7, .5e-7

@time sol = solve(prob, BS5(); abstol=abstol, reltol=reltol)

length(sol)

Now we can plot an animation of the density.

In [ ]:
anim = @animate for t in range(tspan..., step=1/24)
    plot_density(sol(t); legend=false, xrange=(-1, 15), yrange=(0, 1),
        title="Traffic density", xlabel="position", ylabel="density")
end
gif(anim, "plots/traffic2.gif")

We can translate the particles so that the barycenter stays at the origin.

In [ ]:
barycenter(x::AbstractVector) = (sum(x) - (x[1] + x[end]) / 2) / (length(x) - 1)
recenter(x::AbstractVector) = x .- barycenter(x)

In [ ]:
anim = @animate for t in range(tspan..., step=1/24)
    plot_density(recenter(sol(t)); legend=false, xrange=(-3, 4), yrange=(0, 1),
        title="Traffic density (centered)", xlabel="position", ylabel="density")
end
gif(anim, "plots/traffic_centered2.gif")